In [5]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import scipy.stats as stats

plt.rcParams.update({'font.size': 14})
pd.set_option('display.float_format', lambda x: '%.2f' % x)
path = '../../../../playwright/results/core-web-vitals/testrun-8/'


In [6]:
df = pd.read_csv(f'{path}2024-12-17T11:09:51.655Z_results-ssrh-sd.csv', sep=',')
df_t = pd.read_csv(f'{path}2024-12-17T12:56:03.620Z_results-ssrh-sd-throttled.csv', sep=',')
df_ht = pd.read_csv(f'{path}2024-12-17T14:52:01.359Z_results-ssrh-sd-throttled-heavy.csv', sep=',')

features = ['navTime', 'totalTime', 'lcp', 'fcp', 'ttfb', 'tbt', 'tti', 'longestTask', 'longTasks', 'nf:init', 'nf:config','nf:loaded']

dirty_dfs = {
    'unthrottled': df[5:],
    'throttled': df_t[5:],
    'heavy': df_ht[5:],
}

In [7]:
def detect_outliers(_df, _features, contamination=0.1):
    clf = IsolationForest(contamination=contamination, random_state=42)
    outliers = clf.fit_predict(_df[_features])
    return outliers == 1

masks = {}
dfs = {}
target_features = ['navTime', 'totalTime', 'lcp', 'fcp', 'ttfb']

for name, _df in dirty_dfs.items():
    mask = detect_outliers(_df, target_features)
    masks[name] = mask
    dfs[name] = _df[mask].copy()

In [8]:
columns = [ 'ttfb','fcp','nf:init','lcp','tti','nf:loaded','tbt','longestTask']
rows = []

for name, df in dfs.items():
    mean_row = df[columns].mean()
    percentile_row = df[columns].quantile(0.75)
    
    rows.append((f"{name} (mean)", mean_row))
    rows.append((f"{name} (75th)", percentile_row))

result_df = pd.DataFrame([row[1] for row in rows], index=[row[0] for row in rows])
result_df = result_df.mask(result_df < 0, '-')
result_df

,ttfb,fcp,nf:init,lcp,tti,nf:loaded,tbt,longestTask
unthrottled (mean),16.36,90.62,53.95,90.62,90.62,129.88,0.00,-
unthrottled (75th),17.17,92.70,56.00,92.70,92.70,132.70,0.00,-
throttled (mean),369.62,1190.44,1482.99,1190.44,5283.62,5297.43,31.26,81.26
throttled (75th),370.60,1194.20,1487.48,1194.20,5293.50,5303.10,41.00,91.00
heavy (mean),370.93,1260.22,1590.14,1260.22,5707.45,5713.33,161.15,204.42
heavy (75th),371.60,1263.50,1594.75,1263.50,5715.58,5719.98,182.00,232.00
